#### 현재 상태

In [14]:
import pandas as pd
import numpy as np
import riskfolio as rp
import pprint
from datetime import datetime

df = pd.read_excel(r'C:\Users\user\Desktop\RP WS\리스크패리티데이터.xlsx')
df = df.dropna()
df = df.set_index('Name')

In [15]:
df

,KBSTAR 중기우량회사채,KOSEF 코스피100,TIGER 미국S&P500,KODEX 미국채10년선물,TIGER 원유선물Enhanced(H)
Name,,,,,
2020-08-07,101832,22692.0,9685.0,12380.0,2060.0
2020-08-10,101755,22988.0,9709.0,12395.0,2050.0
2020-08-11,101813,23345.0,9719.0,12385.0,2060.0
2020-08-12,101876,23387.0,9680.0,12325.0,2055.0
2020-08-13,101939,23514.0,9728.0,12325.0,2070.0
...,...,...,...,...,...
2023-09-11,103565,25470.0,14905.0,10960.0,4820.0
2023-09-12,103590,25370.0,14905.0,10940.0,4840.0
2023-09-13,103540,25390.0,14835.0,10960.0,4905.0


In [16]:
Y = df.iloc[:, :].pct_change().dropna()
def annualized_return(daily_return,n):
    return(1+daily_return)**n - 1

Y_annualized = annualized_return(Y,250)
Y_annualized

,KBSTAR 중기우량회사채,KOSEF 코스피100,TIGER 미국S&P500,KODEX 미국채10년선물,TIGER 원유선물Enhanced(H)
Name,,,,,
2020-08-10,-0.172303,24.533091,0.856603,0.353542,-0.703749
2020-08-11,0.153105,46.116438,0.293511,-0.182722,2.375513
2020-08-12,0.167246,0.567326,-0.634031,-0.703019,-0.455307
2020-08-13,0.167135,2.872572,2.443905,0.000000,5.160696
2020-08-14,-0.057167,-0.963769,1.718983,-0.398070,0.000000
...,...,...,...,...,...
2023-09-11,-0.242286,0.894246,0.398896,-0.744589,5.184065
2023-09-12,0.062199,-0.625992,0.000000,-0.366580,1.815664
2023-09-13,-0.113698,0.217751,-0.691757,0.578731,27.080997


In [17]:
# 현재 weight
current_quantity = [602, 300, 436, 1763, 731]
band = 0.1


total_money = 10000000
w_s = []
for i in range(len(Y.columns)):
    w_s.append(current_quantity[i]*df.iloc[-1][i])
    total_money += current_quantity[i]*df.iloc[-1][i]

for i in range(len(Y.columns)):
    w_s[i] = (w_s[i]/total_money)

In [26]:
w_s

[0.5686622056294671,
 0.07133880060581926,
 0.059548894877487664,
 0.17579240345222555,
 0.03351319485263256]

In [18]:
def Optimal_portfolio_weight(Y):
    # Building the portfolio object
    port = rp.Portfolio(returns=Y)

    # Calculating optimal portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical) or FM (Factor Model)
    rm = 'MV' # Risk measure used, this time will be variance
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    b = None # Risk contribution constraints vector
    w_rp = port.rp_optimization(model=model, rm=rm, rf=rf, b=b, hist=hist)

    return(w_rp.T.values[0])

In [19]:
def _allocation_risk(weights, covariances):

    # We calculate the risk of the weights distribution
    portfolio_risk = np.dot(weights,np.dot(covariances, weights.T))

    # It returns the risk of the weights distribution
    return portfolio_risk

def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    # We calculate the risk of the weights distribution
    portfolio_risk = _allocation_risk(weights, covariances)

    # We calculate the contribution of each asset to the risk of the weights
    # distribution
    assets_risk_contribution = []
    for i in range(0,len(Y.columns)):
        assets_risk_contribution.append(weights[i]*(np.dot(covariances, weights.T) / portfolio_risk)[i])

    # It returns the contribution of each asset to the risk of the weights
    # distribution
    return assets_risk_contribution

def PCRS(Y, w_s):
    returns = Y * len(Y)
    cov_matrix = returns.cov().values
    weights = np.array(w_s)
    pcr_s = _assets_risk_contribution_to_allocation_risk(weights, cov_matrix)
    return(np.array(pcr_s))

In [20]:
PCRS = PCRS(Y, w_s)
print("현재 각 자산마다의 PCR")
print(PCRS)
print("")
OPW = Optimal_portfolio_weight(Y)
print("최적 자산 배분 비율")
print(OPW)

현재 각 자산마다의 PCR
[0.20013042 0.19997831 0.19981426 0.20026065 0.19981636]

최적 자산 배분 비율
[0.6256397  0.078499   0.06557788 0.19338581 0.03689761]


In [21]:
# NumPy ndarray를 데이터 프레임으로 변환
df_PCRS = pd.DataFrame(PCRS)
df_OPW = pd.DataFrame(OPW)

# Band 설정
Optimal_PCR = 1/len(Y.columns)
PCR_upper_bound = Optimal_PCR * (1+band)
PCR_lower_bound = Optimal_PCR * (1-band)

a = PCR_upper_bound
Pub = pd.DataFrame([a,a,a,a,a])
b = PCR_lower_bound
Plb = pd.DataFrame([b,b,b,b,b])

# 두 데이터프레임을 세로로 붙이기
result_df = pd.concat([Pub, df_PCRS, Plb, df_OPW], axis=1)
result_df = result_df.T
result_df.index = ['Upper PCRS','PCRS','Lower PCRS', 'OPW']
result_df.columns = Y.columns
result_df

,KBSTAR 중기우량회사채,KOSEF 코스피100,TIGER 미국S&P500,KODEX 미국채10년선물,TIGER 원유선물Enhanced(H)
Upper PCRS,0.22000,0.220000,0.220000,0.220000,0.220000
PCRS,0.20013,0.199978,0.199814,0.200261,0.199816
Lower PCRS,0.18000,0.180000,0.180000,0.180000,0.180000
OPW,0.62564,0.078499,0.065578,0.193386,0.036898


#### 리밸런싱 여부 판단

In [22]:
def Rebalancing_decision(result_df):
    decision = (result_df.loc['PCRS'] >= result_df.loc['Lower PCRS']) & (result_df.loc['PCRS'] <= result_df.loc['Upper PCRS'])

    if decision.all():
        return 1
    else:
        return 0

decision = Rebalancing_decision(result_df)

#### 자동매매를 통한 리밸런싱

In [23]:
def rebalancing(result_df):
    # 최적 주식수
    optimized_quantity = []
    for i in range(0, len(Y.columns)):
        optimized_quantity.append(total_money * result_df.iloc[-1][i]/df.iloc[-1][i])
    opt_quantity = [int(x) for x in optimized_quantity]

    # 매수 매도 알고리즘
    sell_buy = []
    sell_buy_quantity = []
    for i in range(0, len(Y.columns)):
        diff = current_quantity[i] - opt_quantity[i]
        if diff > 0:
            sell_buy.append('sell')
            sell_buy_quantity.append(diff)
            # 백테스트시 sell(df.columns[i], diff)

        elif diff < 0:
            sell_buy.append('buy')
            sell_buy_quantity.append(abs(diff))
            # 백테스트시 buy(df.columns[i], abs(diff))

        else:
            sell_buy.append('no')
            sell_buy_quantity.append(0)

    sb = pd.DataFrame(sell_buy)
    sbq = pd.DataFrame(sell_buy_quantity)

    # 두 데이터프레임을 세로로 붙이기
    auto_sell_buy = pd.concat([sb, sbq], axis=1)
    auto_sell_buy = auto_sell_buy.T
    auto_sell_buy.index = ['Sell & Buy & NO', 'Quantity']
    auto_sell_buy.columns = Y.columns
    return(auto_sell_buy)
    

In [24]:
def main():
    if decision == 1:
        print("리밸런싱 필요없음")
    elif decision ==0:
        print("리밸런싱 필요")
        print("")
        auto_sell_buy = rebalancing(result_df)
        auto_sell_buy.to_excel(f'auto_sell_buy_{datetime.today().month}_{datetime.today().day}.xlsx')
        

In [25]:
main()

리밸런싱 필요없음
